In [ ]:
import weave
weave.use_frontend_devmode()

In [ ]:
from weave import syndata
runs = weave.save(syndata.random_runs(10, 10000, 10), 'runs')

In [ ]:
# Doing this unnest operation in the UI doesn't work (like in the varbar). I don't remember
# if the types are wrong, or what the specific situation is.
# TODO: Figure out why we have to do this!
unnested_runs = weave.save(weave.use(runs.unnest()), 'random_runs_unnested')
#unnested_runs

In [ ]:
from weave import weave_internal

plot_count = 0

def plot_for_metric(metric_name):
    # This is a super weird way to make a two column grid layout.
    # Should be the default for boards anyway.
    global plot_count
    even = plot_count % 2 == 0   
    x = 0 if even else 12
    plot_count += 1
    
    return weave.panels.BoardPanel(
            lambda runs, groupby, zoom_x, domain: weave.panels.Plot(
                            runs,
                            # This is an ugly way to declare these multi-series Plots, and I've
                            # had to do some hacking in panel_plot.py to allow it.
                            #
                            # I think it'd be nicer to declare these like vega-lite where you can have
                            # outer values that are consistent across series, and then override
                            # series values with nesting.
                            #
                            # I also am not declaring groupby here, I hacked panel_plot.py to just
                            # always do that for x and label, which obviously can't be merged.
                            #
                            # TODO: fix this stuff
                            series=[
                                weave.panels.Series(
                                        runs,
                                    select_functions={
                                        'x': lambda row: row["history.step"].bin(
                                            weave.ops.numbers_bins_equal(domain, 250))["start"],
                                        'label': lambda row: row[groupby],
                                        'y': lambda row: row["history"][metric_name].avg()
                                    },
                                    constants=weave.panels.PlotConstants(
                                        mark='line'
                                    )
                                ),
                                weave.panels.Series(
                                    runs,
                                    select_functions={
                                        'x': lambda row: row["history.step"].bin(
                                            weave.ops.numbers_bins_equal(domain, 250))["start"],
                                        'label': lambda row: row[groupby],
                                        'y': lambda row: row["history"][metric_name].min(),
                                        'y2': lambda row: row["history"][metric_name].max(),
                                    },
                                    constants=weave.panels.PlotConstants(
                                        mark='area'
                                    )
                                )
                            ],
                            domain_x=zoom_x,
                        ),
            layout=weave.panels.BoardPanelLayout(x=x, y=0, w=12, h=6)
        )

board = weave.panels.Board(
    vars={
        'runs': unnested_runs,
        'groupby': 'config.a',
        # This holds the zoom boundaries, and is two-way synchronized with the plot zooms
        'zoom_x': weave_internal.const(None),
        # This is the extent of the step field in our data (the minimum and maxium values)
        'step_extent': lambda runs: weave.ops.make_list(
            a=weave.ops.numbers_min(runs['history']['step']),
            b=weave.ops.numbers_max(runs['history']['step'])),
        # The "domain" used for binning, which is the zoom range if its non-null, otherwise
        # the step_extent
        'domain': lambda zoom_x, step_extent: zoom_x.coalesce(step_extent)
    },
    # TODO: we're going to want to use an EachColumn for customer setups
    panels=[
        plot_for_metric('metric0'),
        plot_for_metric('metric1'),
        plot_for_metric('metric2'),
        plot_for_metric('metric3'),
        plot_for_metric('metric4'),
        plot_for_metric('metric5'),
        plot_for_metric('metric6'),
        plot_for_metric('metric7'),
    ]
)

In [ ]:
board

In [ ]:
# # single series formulation

# from weave import weave_internal
# board = weave.panels.Board(
#     vars={
#         'runs': unnested_runs,
#         'groupby': 'config.a',
#         'zoom_x': weave_internal.const(None),
#         'step_extent': lambda runs: weave.ops.make_list(
#             a=weave.ops.numbers_min(runs['history']['step']),
#             b=weave.ops.numbers_max(runs['history']['step'])),
#         'domain': lambda zoom_x, step_extent: zoom_x.coalesce(step_extent)
#     },
#     panels=[
#         weave.panels.BoardPanel(
#             lambda runs, groupby, zoom_x, domain: weave.panels.Plot(
#                             runs,
#                             x=lambda row: row["history.step"].bin(
#                                 weave.ops.numbers_bins_equal(domain, 250))["start"],
#                             label=lambda row: row[groupby],
#                             # TODO: I hard coded grouping
#                             #groupby_x=True,
#                             y=lambda row: row["history"]["metric0"].avg(), 
#                             #groupby_label=True,
#                             mark="line",
#                             domain_x=zoom_x,
#                         ),
#             layout=weave.panels.BoardPanelLayout(x=0, y=0, w=24, h=6)
#         ),
#         weave.panels.BoardPanel(
#             lambda runs, groupby, zoom_x, domain: weave.panels.Plot(
#                             runs,
#                             x=lambda row: row["history.step"].bin(
#                                 weave.ops.numbers_bins_equal(domain, 250))["start"],
#                             label=lambda row: row[groupby],
#                             #groupby_x=True,
#                             y=lambda row: row["history"]["metric1"].avg(),
#                             #groupby_label=True,
#                             mark="line",
#                             domain_x=zoom_x,
#                         ),
#             layout=weave.panels.BoardPanelLayout(x=0, y=0, w=24, h=6)
#         ),
#         weave.panels.BoardPanel(
#             lambda runs, groupby, domain: weave.panels.Plot(
#                             runs,
#                             x=lambda row: row["history.step"].bin(
#                                 weave.ops.numbers_bins_equal(domain, 250))["start"],
#                             label=lambda row: row[groupby],
#                             #groupby_x=True,
#                             y=lambda row: row["history"]["metric2"].avg(),
#                             #groupby_label=True,
#                             mark="line",
#                             domain_x=domain,
#                         ),
#             layout=weave.panels.BoardPanelLayout(x=0, y=0, w=24, h=6)
#         ),
#         weave.panels.BoardPanel(
#             lambda runs, groupby, domain: weave.panels.Plot(
#                             runs,
#                             x=lambda row: row["history.step"].bin(
#                                 weave.ops.numbers_bins_equal(domain, 250))["start"],
#                             label=lambda row: row[groupby],
#                             #groupby_x=True,
#                             y=lambda row: row["history"]["metric3"].avg(),
#                             #groupby_label=True,
#                             mark="line",
#                             domain_x=domain,
#                         ),
#             layout=weave.panels.BoardPanelLayout(x=0, y=0, w=24, h=6)
#         ),
#     ]
# )